## OCT Image Interface

In [1]:
import tkinter as tk
from PIL import ImageTk, Image
import abc
import pandas as pd

In [2]:
class GUIController():
    
    def __init__(self, view, image_model, patient_info_model):
        self.view = view
        self.image_model = image_model
        self.patient_info_model = patient_info_model
        
        self.check_list = self.fetch_check_list_info()
        
    def fetch_check_list_info(self):
        return ["Vitreous or Subhyaloid Space", 
                "Posterior Hyaloid", 
                "Epiretinal Membrane",
                "Unknown"]

In [3]:
# abstract image class/image interface
class ImageAbstraction(abc.ABC):
    
    @abc.abstractmethod
    def class_name():
        """prints name of class"""
    
    @abc.abstractmethod
    def top_image(filename, width, height):
        """retrieve top image"""
    
    @abc.abstractmethod
    def side_image(filename, width, height):
        """retrieve side image"""
    
    @abc.abstractmethod
    def number_of_images():
        """retreive total number of images"""
    
    @abc.abstractmethod
    def image_dimensions():
        """retrieve image dimensions"""

        
class EyeImages(ImageAbstraction):
    
    def class_name():
        return "EyeImages"
    
    def get_top_image(filename = "eye.png", width = 100, height = 100):
        img = Image.open("eye.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_side_image(filename = "eye.png", width = 100, height = 100):
        img = Image.open("eye.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_number_of_images():
        return 10
    
    def get_image_dimensions():
        return 100,100
    
    
class CatImages(object):
    
    def class_name():
        return "CatImages"
    
    def get_top_image(filename = "cat.png", width = 50, height = 50):
        img = Image.open("cat.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_side_image(filename = "cat.png", width = 50, height = 50):
        img = Image.open("cat.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_number_of_images():
        return 5
    
    def get_image_dimensions():
        return 50,50


In [4]:
class PatientInfoRetrieverAbstraction(abc.ABC):
    
    @abc.abstractmethod
    def load_patient_data():
        """ open/load file """
      
    @abc.abstractmethod
    def next_patient():
        """ move pointer to next patient """
        
    @abc.abstractmethod
    def previous_patient():
        """ move pointer to previous patient """
        
    @abc.abstractmethod
    def get_id():
        """ return ID of current patient """
        
    @abc.abstractmethod
    def get_name():
        """ return name of current patient """
        
    @abc.abstractmethod
    def get_age():
        """ return age of current patient """
        
class PatientInfoRetrieverCSV(PatientInfoRetrieverAbstraction):
    
    def __init__(self, filename):
        self.filename = filename
    
    def load_patient_data(self, init_index = 0):
        self.patient_table = pd.read_csv(self.filename)
        self.current_index = init_index
        self.current_patient = self.patient_table.loc[init_index]
      
    def next_patient(self):
        try:
            self.current_index += 1
            self.current_patient = self.patient_table.loc[self.current_index]
        except KeyError:
            print("You have reached the end of the file.")
        
    def previous_patient(self):
        try:
            self.current_index -= 1
            self.current_patient = self.patient_table.loc[self.current_index]
        except KeyError:
            print("You have reached the beginning of the file.")
        
    def get_id(self):
        return self.current_patient["PatientID"]
        
    def get_name(self):
        return self.current_patient["Name"]
        
    def get_age(self):
        return self.current_patient["Age"]

In [9]:
class PatientInformation(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent 
        
        # TODO: SHOULD NOT BE DOING THIS HERE
        db = parent.controller.patient_info_model("patient_data_table.csv")
        db.load_patient_data()
        
        patient_info_string = ("Name: " + db.get_name() + "\n" 
                               + "ID: " + str(db.get_id()) + "\n" 
                               + "Age: " + str(db.get_age()) + "\n")
        patient_info_label = tk.Label(self, text = patient_info_string, anchor = tk.W, background = '#FFFFFF', justify=tk.LEFT)
        patient_info_label.grid(row=0, column=0, sticky = tk.E)
        
class ImageScroller(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent 
        image_model = parent.controller.image_model
        
        # first image
        image_width, image_height = image_model.get_image_dimensions()
        img_data = image_model.get_top_image()
        img = ImageTk.PhotoImage(img_data)
        self.img = img # save images to prevent garbage collection
        w1 = tk.Label(self, image=self.img)
        w1.grid(row=0, column=1, sticky=tk.W)
        
        # scrollable images
        number_of_images = image_model.get_number_of_images()
        full_scroll_width = number_of_images*image_width
        canvas = tk.Canvas(self, bg='#FFFFFF', width=image_width, height=image_height, 
                        scrollregion=(0,0,full_scroll_width,image_height))
        width_offset = 0
        for i in range(number_of_images):
            canvas.create_image(width_offset, 0, image=img, anchor=tk.NW)
            width_offset += image_width

        # scrollbar
        hbar=tk.Scrollbar(self, orient=tk.HORIZONTAL)
        hbar.grid(row=1, column=0, sticky=tk.N+tk.S+tk.E+tk.W, padx=10, pady=10)
        hbar.config(command=canvas.xview)
        canvas.config(xscrollcommand=hbar.set)
        canvas.grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)

        
class DropdownMenu(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent 

        tk.Label(self, text="Field:").grid(row=4)
        tk.Label(self, text="Contents:").grid(row=5)
        self.contentsVar = tk.StringVar()
        self.contentsVar.set(" ")
        
        self.contents_entry_box = tk.Entry(self, textvariable=self.contentsVar)
        self.contents_entry_box.grid(row=5, column=1)
        
        save_button = tk.Button(self, text="Save", command=lambda: print(self.contents_entry_box.get()))
        save_button.grid(row=5, column=2, padx=10, pady=10, sticky=tk.W)
        
        self.field_to_contents = {'Name': 'Jane Doe', 'Age': 50, 'Diagnosis': 'M17.11'}
        fieldVar = tk.StringVar(self)
        fieldVar.set("Select a field to edit") # default value
        self.field_dropdown = tk.OptionMenu(self, fieldVar, *self.field_to_contents, command=self.displayContents)
        self.field_dropdown.grid(row=4, column=1)
        
    def displayContents(self, value): # selected option implicitly passed into this function
        self.contentsVar.set(self.field_to_contents[value])
        
class CheckList(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent 
        
        self.listbox = tk.Listbox(self, selectmode = "multiple" )
        
        self.check_list = parent.controller.check_list
        for item in self.check_list:
            self.listbox.insert(tk.END, item)
        self.listbox.grid(row=0, column=0)
        
        save_button = tk.Button(self, text="Save", command=self.get_all_selected)
        save_button.grid(row=0, column=1, padx=10, pady=10, sticky=tk.W)
        
    def get_all_selected(self):
        values = [self.listbox.get(idx) for idx in self.listbox.curselection()]
        print(', '.join(values))
        

class StartPage(tk.Frame):
    def __init__(self, parent, controller, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent 
        self.controller = controller
        
        self.patient_info = PatientInformation(self)
        self.image_scroller = ImageScroller(self)
        self.dropdown_menu = DropdownMenu(self)
        self.check_list = CheckList(self)

        self.patient_info.pack()
        self.image_scroller.pack()
        self.dropdown_menu.pack() 
        self.check_list.pack()

In [10]:
if __name__ == "__main__":
    root = tk.Tk()
    root.withdraw()
    top_level = tk.Toplevel()
    top_level.title('Image Annotation Interface')
    top_level.protocol('WM_DELETE_WINDOW', root.destroy) # override 'X' button
    controller = GUIController(StartPage, EyeImages, PatientInfoRetrieverCSV) # configure GUI controller
    StartPage(top_level, controller).pack(side="top", fill="both", expand=True)
    root.mainloop()

In [ ]:
from tkinter import *
from PIL import ImageTk, Image

In [ ]:
class ImageAnnotationGUI(object):
    
    PADX = 10
    PADY = 10
    
    def __init__(self, controller, image_model):
        self.controller = controller
        self.image_model = image_model
    
    def run(self):
        
        # construct window
        self.base_window = Tk() # must run this window, but contains no contents
        self.base_window.withdraw() # hide the window with no contents 
        self.root = Toplevel() # this is the main window (must use Toplevel to display images)
        base_window = self.base_window # makes code easier to read going onward
        root = self.root
        root.title('Image Annotation Interface')
        root.protocol ("WM_DELETE_WINDOW", self.destroy_all) # override 'X' button
        
        # open window in full screen mode
        screen_padding = 3
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        root.geometry("{0}x{1}+0+0".format(
            screen_width - screen_padding, 
            screen_height - screen_padding)
        )
        
        # create frame to hold contents
        f = Frame(root)
        f.pack(padx = self.PADX, pady = self.PADY)
        
        # Patient information
        patient_info_string = "PATIENT NAME\n" + "ID NUMBER\n" + "AGE\n"
        patient_info_label = Label(f, text = patient_info_string, anchor = W, background = '#FFFFFF', justify=LEFT)
        patient_info_label.grid(row=0, column=0, sticky = E)
        
        # first image
        image_width, image_height = self.image_model.get_image_dimensions()
        img = self.image_model.get_top_image()
        photoImg =  ImageTk.PhotoImage(img)
        w1 = Label(f, image=photoImg)
        w1.grid(row=1, column=1, sticky=W)
        
        # scrollable images
        number_of_images = self.image_model.get_number_of_images()
        full_scroll_width = number_of_images*image_width
        canvas = Canvas(f, bg='#FFFFFF', width=image_width, height=image_height, 
                        scrollregion=(0,0,full_scroll_width,image_height))
        width_offset = 0
        for i in range(number_of_images):
            canvas.create_image(width_offset, 0, image=photoImg, anchor=NW)
            width_offset += image_width

        # scrollbar
        hbar=Scrollbar(f,orient=HORIZONTAL)
        hbar.grid(row=2, column=0, sticky=N+S+E+W, padx=10, pady=10)
        hbar.config(command=canvas.xview)
        canvas.config(xscrollcommand=hbar.set)
        canvas.grid(row=1, column=0, padx=10, pady=10, sticky=W)
        
        # dropdown
        Label(f, text="Field:").grid(row=4)
        Label(f, text="Contents:").grid(row=5)
        self.contentsVar = StringVar()
        self.contentsVar.set(" ")
        
        self.contents_entry_box = Entry(f, textvariable=self.contentsVar)
        self.contents_entry_box.grid(row=5, column=1)
        
        save_button = Button(f, text="Save", command=lambda: print(contents_entry_box.get()))
        save_button.grid(row=5, column=2, padx=10, pady=10, sticky=W)
        
        self.field_to_contents = {'Name': 'Jane Doe', 'Age': 50, 'Diagnosis': 'M17.11'}
        fieldVar = StringVar(f)
        fieldVar.set("Select a field to edit") # default value
        self.field_dropdown = OptionMenu(f, fieldVar, *self.field_to_contents, command=self.displayContents)
        self.field_dropdown.grid(row=4, column=1)
        
        # run interface
        mainloop()
        
    def displayContents(self, value): # selected option implicitly passed into this function
        self.contentsVar.set(self.field_to_contents[value])
        
    def destroy_all(self):
        self.base_window.destroy()

### Image Modules

In [ ]:
import abc

# abstract image class/image interface
class ImageAbstraction(abc.ABC):
    
    @abc.abstractmethod
    def class_name():
        """prints name of class"""
    
    @abc.abstractmethod
    def top_image(filename, width, height):
        """retrieve top image"""
    
    @abc.abstractmethod
    def side_image(filename, width, height):
        """retrieve side image"""
    
    @abc.abstractmethod
    def number_of_images():
        """retreive total number of images"""
    
    @abc.abstractmethod
    def image_dimensions():
        """retrieve image dimensions"""

        
class EyeImages(ImageAbstraction):
    
    def class_name():
        return "EyeImages"
    
    def get_top_image(filename = "eye.png", width = 100, height = 100):
        img = Image.open("eye.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_side_image(filename = "eye.png", width = 100, height = 100):
        img = Image.open("eye.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_number_of_images():
        return 10
    
    def get_image_dimensions():
        return 100,100
    
    
class CatImages(object):
    
    def class_name():
        return "CatImages"
    
    def get_top_image(filename = "cat.png", width = 50, height = 50):
        img = Image.open("cat.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_side_image(filename = "cat.png", width = 50, height = 50):
        img = Image.open("cat.png")
        return img.resize((width, height), Image.ANTIALIAS)
    
    def get_number_of_images():
        return 5
    
    def get_image_dimensions():
        return 50,50
    
    

### Display Modules

In [ ]:
# display modules will go here

### Controller 
(Module chooser)

In [ ]:
# Controlller that makes calls to the database for us,
# and hopefully makes it so I won't have to make too many 
# changes to ImageAnnotationGUI as I'm figuring out how to 
# get the data from the database

# Also hopefully makes it so that we can easily trade 
# OCT images for another modality
class GUIController(object):
    
    def __init__(self, image_model, view):
        self.view = view
        self.image_model = image_model
    
    # runs view
    def start(self):
        window = self.view(self, self.image_model)
        window.run()
    


if __name__ == "__main__":
    # needs to be some sort of protocol here that sets the preferences for the controller
    gui_controller = GUIController(EyeImages, ImageAnnotationGUI)
    gui_controller.start()

In [ ]:
from tkinter import *
from PIL import ImageTk, Image

class ImageAnnotationGUI(object):
    
    def run(self):
        
        # window set-upI
        base_window = Tk()
        base_window.withdraw()
        self.base_window = base_window
        root = Toplevel()
        self.root = root
        
        #root.geometry('768x612')
        pad=3
        root.geometry("{0}x{1}+0+0".format(
        root.winfo_screenwidth()-pad, root.winfo_screenheight()-pad))
        
        root.title('Image Annotation Interface')
        f = Frame(root)
        f.pack(padx = 10, pady = 10)
        root.protocol ("WM_DELETE_WINDOW", self.destroy_all)
        
        # create QUIT button
        button = Button(f, text="QUIT", fg="red", command=self.destroy_all)
        button.grid(row=0, column=0, padx=10, pady=10, sticky=W)

        # create Hello button
        hi_there = Button(f, text="Hello", command=self.say_hi)
        hi_there.grid(row=0, column=1, padx=10, pady=10, sticky=W)
 
        # create horizontal scroll bar
        xscrollbar = Scrollbar(f, orient=HORIZONTAL)
        xscrollbar.grid(row=2, column=0, sticky=N+S+E+W, padx=10, pady=10)
        text = Text(f, wrap=NONE, xscrollcommand=xscrollbar.set, height=1, width=100)
        text.grid(row=1, column=0)
        xscrollbar.config(command=text.xview)
        scroll_contents = [i for i in range(200)]
        text.insert(END, scroll_contents)
        
        # create first label
        first_label = Label(f, text="First")
        first_label.grid(row=3, sticky=E)
        first_entry_box = Entry(f)
        first_entry_box.grid(row=3, column=1)
        
        # create first label save button
        first_save_button = Button(f, text="Save First Box", command=lambda: print(first_entry_box.get()))
        first_save_button.grid(row=3, column=2, padx=10, pady=10, sticky=W)
        
        # create second label
        second_label = Label(f, text="Second")
        second_label.grid(row=4, sticky=E)
        second_entry_box = Entry(f)
        second_entry_box.grid(row=4, column=1)
        
        # create second label save button
        second_save_button = Button(f, text="Save Second Box", command=lambda: print(second_entry_box.get()))
        second_save_button.grid(row=4, column=2, padx=10, pady=10, sticky=W)
        
        # first image
        width = 50
        height = 50
        img = Image.open("cat.png")
        img = img.resize((width,height), Image.ANTIALIAS)
        photoImg =  ImageTk.PhotoImage(img)
        w1 = Label(f, image=photoImg)
        w1.grid(row=5, column=0)
        
        # another scroll bar
        canvas=Canvas(f,bg='#FF00FF',width=300,height=300,scrollregion=(0,0,500,500))
        canvas.create_image(0, height, image=photoImg, anchor=NW)
        canvas.create_image(width*2, height, image=photoImg, anchor=NW)
        canvas.create_image(width*4, height, image=photoImg, anchor=NW)
        canvas.create_image(width*6, height, image=photoImg, anchor=NW)
        canvas.create_image(width*8, height, image=photoImg, anchor=NW)
        hbar=Scrollbar(f,orient=HORIZONTAL)
        hbar.grid(row=7, column=0, sticky=N+S+E+W, padx=10, pady=10)
        hbar.config(command=canvas.xview)
        canvas.config(width=300,height=300)
        canvas.config(xscrollcommand=hbar.set)
        canvas.grid(row=6, column=0, padx=10, pady=10)
        
        # run interface
        mainloop()
        
    def destroy_all(self):
        self.base_window.destroy()

        
    def say_hi(self):
        print("hi there, everyone!")

if __name__=='__main__':
    window = ImageAnnotationGUI()
    window.run()